## Objectives

- Run the ETL process
- Extract bank and market cap data from the JSON file `bank_market_cap.json`
- Transform the market cap currency using the exchange rate data
- Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries


In [1]:
!mamba install pandas==1.3.3 -y
!mamba install requests==2.26.0 -y


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.2) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['pandas==1.3.3']

[+] 0.0s
[+] 0.1s
pkgs/main/linux-64 ╸━━━━━━━━━━━━━━━╸━━━━━━━━   0.0 B /  ??.?MB 

In [2]:
import glob
import pandas as pd
from datetime import datetime

As the exchange rate fluctuates, we will download the same dataset to make marking simpler


In [1]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-07-10 20:14:57--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-07-10 20:14:57 (23.1 MB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2023-07-10 20:14:57--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tra

## Extract


### JSON Extract Function

This function will extract JSON files.


In [3]:
def extract_from_json(file_to_process):
    # Read JSON file into a DataFrame
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [18]:
columns=['Name','Market Cap (US$ Billion)']

In [13]:
#extracted_data = pd.DataFrame(columns=columns)
def extract():
    # Define the file path
    file_to_process = "bank_market_cap_1.json"
    
    # Specify the desired columns
    columns = ['Name', 'Market Cap (US$ Billion)']
    
    # Extract data from JSON file into a DataFrame
    extracted_data = extract_from_json(file_to_process)
    
    # Select the desired columns
    extracted_data = extracted_data[columns]
    
    return extracted_data

# Call the extract function
dataframe = extract()

# Print the resulting DataFrame
print(dataframe)

                                       Name  Market Cap (US$ Billion)
0                            JPMorgan Chase                   390.934
1   Industrial and Commercial Bank of China                   345.214
2                           Bank of America                   325.331
3                               Wells Fargo                   308.013
4                   China Construction Bank                   257.399
..                                      ...                       ...
65                             Ping An Bank                    37.993
66                       Standard Chartered                    37.319
67                     United Overseas Bank                    35.128
68                                QNB Group                    33.560
69                              Bank Rakyat                    33.081

[70 rows x 2 columns]


Load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, we will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [15]:
# Load the CSV file as a DataFrame
df = pd.read_csv("exchange_rates.csv", index_col=0)

# Find the exchange rate for British pounds (GBP)
exchange_rate = df.loc["GBP", "Rates"]

# Print the exchange rate
print(exchange_rate)

0.7323984208000001


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file we will find the exchange rate of USD to GBP

1. Changes the `Market Cap (US$ Billion)` column from USD to GBP
2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
3. Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [19]:
def transform(dataframe):
        dataframe['Market Cap (GBP$ Billion)'] = dataframe['Market Cap (US$ Billion)'] * exchange_rate
        dataframe['Market Cap (GBP$ Billion)'] = dataframe['Market Cap (GBP$ Billion)'].round(3)
        dataframe.drop('Market Cap (US$ Billion)', axis=1, inplace=True)
        return dataframe

## Load

We will Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`


In [20]:
def load(targetfile,data_to_load):

    data_to_load.to_csv(targetfile, index=False)


## Logging Function


We will write the logging function <code>log</code> to log your data:


In [22]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')  

## Running the ETL Process


We will Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [23]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

log("Load phase Started")
load('bank_market_cap_gbp.csv',transformed_data)
log("Load phase Ended")

log("ETL Job Ended")

### Extract


We will use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [25]:
extracted_data = extract()

extracted_data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


We will log the data as <code>"Extract phase Ended"</code>


In [28]:
log('Extract phase Ended')

### Transform


We will log the following  <code>"Transform phase Started"</code>


In [29]:
log('Transform phase Started')

We will use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [30]:
transformed_data = transform(extracted_data)

transformed_data.head()

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


Log your data <code>"Transform phase Ended"</code>


In [31]:
log('Transform phase Ended')

### Load


We will log the following `"Load phase Started"`.


In [32]:
log('Load phase Started')

We will call the load function


In [34]:
load('bank_market_cap_gbp.csv',transformed_data)

We will log the following `"Load phase Ended"`.


In [40]:
log('Load phase Ended')
